<a href="https://colab.research.google.com/github/HausReport/ClubRaiders/blob/master/notebooks/InraTour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import itertools
import math
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

In [2]:
home = ['Pleione', (-77 , -146.78125 , -344.125)]

In [3]:
points = [    ['12 Trianguli',  (-77.15625 , -76.8125 , -120.6875)],    #0
              ['HIP 12099',     (-101.90625 , -95.46875 , -165.59375)], #1
              ['HIP 16824',     (-49.21875 , -89.375 , -187.1875)],     #2
              ['HIP 15329',     (-87.84375 , -52 , -167.375)],          #3
              ['Alnath',        (-3.875 , -7.375 , -133.65625)],        #4
              ['HIP 7158',      (-3.78125 , -143.84375 , -33.8125)],    #5
              ['HIP 59382',     (19.03125 , 130.09375 , -11.59375)],    #6
              ['Conn',          (-36.59375 , -9.1875 , -20.5)],         #7
              ['Hermitage',     (-28.75 , 25 , 10.4375)],               #8
              ['LP 389-95',     (98.84375 , 7 , -7.71875)],             #9
          ]

In [4]:
sequence = [4, 9, 6, 8, 7, 5, 0, 3, 1, 2] #list(range( 0,len(points)))

In [56]:

def metric_dist( pt1, pt2):
  d = 0.0
  d += (pt1[0] - pt2[0]) **2
  d += (pt1[1] - pt2[1]) **2
  d += (pt1[2] - pt2[2]) **2
  d = math.sqrt(d)
  return d

def dist( pt1, pt2):
  d = metric_dist(pt1, pt2)
  return d                   #cartesian metric
  #return math.ceil(d/74)    #jump metric


def score(sequence):
  d = 0.0
  slen = len(sequence)
  d+=  dist( home[1], points[sequence[0]][1])
  for i in range(0, slen-1):
    d+= dist( points[sequence[i]][1], points[sequence[i+1]][1])

  d+= dist( home[1], points[sequence[-1]][1])
  return d

def search_for_best(samples):
  i = 0

  slen = len(sequence)
  perms = math.factorial(slen)
  step = math.floor(perms/samples)

  print(f"Searching {samples} solutions of {perms} with step of {step}")

  best = sequence
  bestScore = score(sequence)
  for cur in itertools.permutations(sequence):
   if i % step == 0:
    if i % (100 * step) == 0:
      print(".")
    else:
      print(".", end='')

    sco = score(cur)
    if sco< bestScore:
      best = cur
      bestScore = sco
   i = i + 1

  print("")
  print(str(bestScore))
  print(best)
  return best

1334.5975515058062
(4, 9, 6, 8, 7, 5, 0, 3, 1, 2)

In [66]:
def makeFrame( seq ):
  names = []
  xs = []
  ys = []
  zs = []

  # From home to 0th point
  names.append( home[0])
  xs.append( home[1][0])
  ys.append( home[1][1])
  zs.append( home[1][2])

  # From 0th point to last point
  for i in range(0, len(seq)):
    thePoint = points[seq[i]]
    names.append( thePoint[0])
    xs.append( thePoint[1][0])
    ys.append( thePoint[1][1])
    zs.append( thePoint[1][2])

  # From last point back to home
  names.append( home[0])
  xs.append( home[1][0])
  ys.append( home[1][1])
  zs.append( home[1][2])

  data = { 'names': names, 'xs':xs, 'ys':ys, 'zs':zs} #, 'dists': dists }
  return pd.DataFrame( data)

In [12]:
def getScene():
        return dict(
            xaxis=dict(
                backgroundcolor="rgb(0,0,0)",
                gridcolor="grey",
                showbackground=False,
                zerolinecolor="white", ),
            yaxis=dict(
                backgroundcolor="rgb(0,0,0)",
                gridcolor="grey",
                showbackground=False,
                zerolinecolor="white", ),
            zaxis=dict(
                backgroundcolor="rgb(0,0,0)",
                gridcolor="grey",
                showbackground=False,
                zerolinecolor="white", ),
            aspectratio=dict(x=1, y=1, z=0.7),
            aspectmode="manual"
)
        
def getLayout(theTitle):
        return go.Layout(title=theTitle,
                         scene=getScene(),
                         width=500,
                         height=350,
                         autosize=False,
                         paper_bgcolor='rgb(0,0,0)',
                         plot_bgcolor='rgb(0,0,0)',
                         clickmode='event+select',
                         font=dict(
                             family="Courier New, monospace",
                             size=12,
                             color="#ffffff"),
                         margin=dict(t=25, b=0, l=0, r=0),
)

In [67]:
goodSeq = search_for_best(1000)
df = makeFrame(goodSeq)

fig = go.Figure(data=[go.Scatter3d(
    x=df['xs'],
    y=df['ys'],
    z=df['zs'],
    text=df['names'],
    mode='markers+lines+text'
    )],
    )

fig.update_layout( getLayout("INRA Route") )
fig.show()

df['dists'] = 0
df['dists'] = (df['xs'] - df['xs'].shift(-1)) ** 2
df['dists'] += (df['ys'] - df['ys'].shift(-1)) ** 2
df['dists'] += (df['zs'] - df['zs'].shift(-1)) ** 2
df['dists'] = df['dists']**(1/2)

df = df.round(1)
print(df.to_markdown())
tot = df['dists'].sum().round()
print(f"Total distance: {tot} ly.")

Searching 1000 solutions of 3628800 with step of 3628
.
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
...................................

|    | names        |     xs |     ys |     zs |   dists |
|---:|:-------------|-------:|-------:|-------:|--------:|
|  0 | Pleione      |  -77   | -146.8 | -344.1 |   262.8 |
|  1 | Alnath       |   -3.9 |   -7.4 | -133.7 |   163.2 |
|  2 | LP 389-95    |   98.8 |    7   |   -7.7 |   146.8 |
|  3 | HIP 59382    |   19   |  130.1 |  -11.6 |   117.5 |
|  4 | Hermitage    |  -28.8 |   25   |   10.4 |    46.8 |
|  5 | Conn         |  -36.6 |   -9.2 |  -20.5 |   139.2 |
|  6 | HIP 7158     |   -3.8 | -143.8 |  -33.8 |   132   |
|  7 | 12 Trianguli |  -77.2 |  -76.8 | -120.7 |    53.9 |
|  8 | HIP 15329    |  -87.8 |  -52   | -167.4 |    45.7 |
|  9 | HIP 12099    | -101.9 |  -95.5 | -165.6 |    57.3 |
| 10 | HIP 16824    |  -49.2 |  -89.4 | -187.2 |   169.4 |
| 11 | Pleione      |  -77   | -146.8 | -344.1 |   nan   |
Total distance: 1335.0 ly.
